<a href="https://colab.research.google.com/github/paritoshkc/TextAnalytics/blob/master/Review_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import re



# Get movie name and year list from wikipedia 
url_wiki='https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
to_year=2010

#Create a handle, page, to handle the contents of the website
page = requests.get(url_wiki).text
soup= BeautifulSoup(page, "lxml")

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.find_all('a')

# for future use
name=[]
year=[]


# used for constrcution of urls
url_front = 'https://www.rottentomatoes.com/m/'
url_back_critics='/reviews'
url_back_audi='/reviews?type=user'

url_critics=[]
url_audi=[]
url_mid=[]


i=0
texts=[]
for link in links:
  texts.append(link.text.split('\n'))
for text in texts:
  if text[0]=='[1]':
    continue
  if i%2==0:
    text_string=text[0].replace(' ','_').lower()
    name.append(text_string)
  else:
    year.append(text[0])
    if int(text[0])<to_year:
      break
  i+=1

zippedList =  list(zip(name,year))

for n,y in zippedList:
  url_mid.append(n+'_'+y)

for mid in url_mid:
  url_critics.append(url_front+mid+url_back_critics)
  url_audi.append(url_front+mid+url_back_audi)




In [7]:
url_critics

['https://www.rottentomatoes.com/m/parasite_2019/reviews',
 'https://www.rottentomatoes.com/m/ford_v_ferrari_2019/reviews',
 "https://www.rottentomatoes.com/m/learning_to_skateboard_in_a_warzone_(if_you're_a_girl)_2019/reviews",
 "https://www.rottentomatoes.com/m/the_neighbors'_window_2019/reviews",
 'https://www.rottentomatoes.com/m/little_women_2019/reviews',
 'https://www.rottentomatoes.com/m/marriage_story_2019/reviews',
 'https://www.rottentomatoes.com/m/jojo_rabbit_2019/reviews',
 'https://www.rottentomatoes.com/m/toy_story_4_2019/reviews',
 'https://www.rottentomatoes.com/m/joker_2019/reviews',
 'https://www.rottentomatoes.com/m/once_upon_a_time_in_hollywood_2019/reviews',
 'https://www.rottentomatoes.com/m/1917_2019/reviews',
 'https://www.rottentomatoes.com/m/judy_2019/reviews',
 'https://www.rottentomatoes.com/m/bombshell_2019/reviews',
 'https://www.rottentomatoes.com/m/rocketman_2019/reviews',
 'https://www.rottentomatoes.com/m/american_factory_2019/reviews',
 'https://www.

In [23]:

user_review=[]
critic_review=[]

def get_user_reviews(url):
    ur_review=[]
  
    print(url)
    response = requests.get(url)
    if response.status_code ==200:
      soup = BeautifulSoup(response.text, "html.parser")
      # audience reviews
      audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
      if len(audience_reviews)!=0 :
        for reviews in audience_reviews:
          # print(reviews.text)
          ur_review.append(reviews.text+' %% ')
    return ur_review


def get_ctric_reviews(url):

  response = requests.get(url)
  crtc_review=[]
  if response.status_code ==200:
    soup = BeautifulSoup(response.text, "html.parser")
    c_reviews=soup.findAll('div', attrs={'class': 'the_review'})
    if len(c_reviews)!=0 :
      for reviews in c_reviews:
        crtc_review.append(reviews.text.replace('                                   ','').replace('\n','').replace('                                ','')+'$$' )
  return crtc_review



  
def get_review_year(year):
  for user_url,critic_url in zip(url_audi,url_critics):
    if year in user_url:
      user_review.append(get_ctric_reviews(critic_url))
      critic_review.append(get_user_reviews(user_url))

get_review_year('2019')

zippedList =  list(zip(url_mid,url_critics,url_audi,user_review,critic_review))
dfObj = pd.DataFrame(zippedList, columns = ['Name' , 'url_critics','url_audi','user_review','critic_review']) 



https://www.rottentomatoes.com/m/parasite_2019/reviews?type=user
https://www.rottentomatoes.com/m/ford_v_ferrari_2019/reviews?type=user
https://www.rottentomatoes.com/m/learning_to_skateboard_in_a_warzone_(if_you're_a_girl)_2019/reviews?type=user
https://www.rottentomatoes.com/m/the_neighbors'_window_2019/reviews?type=user
https://www.rottentomatoes.com/m/little_women_2019/reviews?type=user
https://www.rottentomatoes.com/m/marriage_story_2019/reviews?type=user
https://www.rottentomatoes.com/m/jojo_rabbit_2019/reviews?type=user
https://www.rottentomatoes.com/m/toy_story_4_2019/reviews?type=user
https://www.rottentomatoes.com/m/joker_2019/reviews?type=user
https://www.rottentomatoes.com/m/once_upon_a_time_in_hollywood_2019/reviews?type=user
https://www.rottentomatoes.com/m/1917_2019/reviews?type=user
https://www.rottentomatoes.com/m/judy_2019/reviews?type=user
https://www.rottentomatoes.com/m/bombshell_2019/reviews?type=user
https://www.rottentomatoes.com/m/rocketman_2019/reviews?type=us

In [24]:
critic_review

[['A must see movie from the critticly aclaimed director, Bong joo-ho %% ',
  'very very very good. %% ',
  "This classic tale -- a struggling working-class family taking outrageous advantage to experience life on the other side, with sudden unforeseeable consequences -- glides along effortlessly without relying on banal, been-there devices or silly twists.  It's smart, original, emotionally intelligent, and unsparing.  And, it mixes genres, so be prepared for things not to go the way you think they will. %% ",
  'Horrible. Not worth seeing. The acting was good but the story is garbage and the characters are despicable. I thought this was a film about the contrast of classes but it’s message was lost in the horrific actions. I can’t believe this one for beat picture.  %% ',
  'I loved this movie - best movie Ive seen in a long time! %% ',
  'Just as you think that you have figured out where this is going and you start to get a little bit bored it turns the table over and out of nowhere

In [25]:
dfObj

,Name,url_critics,url_audi,user_review,critic_review
0,parasite_2019,https://www.rottentomatoes.com/m/parasite_2019...,https://www.rottentomatoes.com/m/parasite_2019...,[ Parasite surprised me over and over. I const...,[A must see movie from the critticly aclaimed ...
1,ford_v_ferrari_2019,https://www.rottentomatoes.com/m/ford_v_ferrar...,https://www.rottentomatoes.com/m/ford_v_ferrar...,[ ...it happens to be great filmmaking and fea...,[So emotional. Best ending of a movie I seen i...
2,learning_to_skateboard_in_a_warzone_(if_you're...,https://www.rottentomatoes.com/m/learning_to_s...,https://www.rottentomatoes.com/m/learning_to_s...,[],[]
3,the_neighbors'_window_2019,https://www.rottentomatoes.com/m/the_neighbors...,https://www.rottentomatoes.com/m/the_neighbors...,[],[]
4,little_women_2019,https://www.rottentomatoes.com/m/little_women_...,https://www.rottentomatoes.com/m/little_women_...,[ What's enjoyable about this version of Littl...,"[A good movie for wife and teenage kids %% , P..."
5,marriage_story_2019,https://www.rottentomatoes.com/m/marriage_stor...,https://www.rottentomatoes.com/m/marriage_stor...,[ Baumbach's approach to the challenge of repr...,[in a marriage story no one is bad and no one ...
6,jojo_rabbit_2019,https://www.rottentomatoes.com/m/jojo_rabbit_2...,https://www.rottentomatoes.com/m/jojo_rabbit_2...,"[ Playful, funny and relevant to the world we ...",[It put together comedy and drama into a film ...
7,toy_story_4_2019,https://www.rottentomatoes.com/m/toy_story_4_2...,https://www.rottentomatoes.com/m/toy_story_4_2...,[],[]
8,joker_2019,https://www.rottentomatoes.com/m/joker_2019/re...,https://www.rottentomatoes.com/m/joker_2019/re...,[ It's disingenuous and pandering for Phillips...,[It helps to know the Joker if you're a fan of...
9,once_upon_a_time_in_hollywood_2019,https://www.rottentomatoes.com/m/once_upon_a_t...,https://www.rottentomatoes.com/m/once_upon_a_t...,[ This is often surprisingly funny in a playfu...,[Best movie of the year. It was hilarious. Bes...


In [0]:

# def get_user_reviews():
#     user_review=[]
#     for url in url_audi:
#       if '2019' in url:
#         print(url)
#         response = requests.get(url)

#         soup = BeautifulSoup(response.text, "html.parser")


#         # audience reviews
#         audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
#         if len(audience_reviews)!=0 :
#           for reviews in audience_reviews:
#             # print(reviews.text)
#             user_review.append(reviews.text+' %% ')
#     return user_review
      


      # def get_ctric_reviews():
#   critic_review=[]
#   review_pat = re.compile(r'<div class=\"the_review\">[;a-zA-Z\s,-.\'\/\?\[\]\":\']*</div>')


#   for url in url_critics:
#     if '2019' in url:
#       print(url)
#       response = requests.get(url)
#       soup = BeautifulSoup(response.text, "html.parser")

#       soup = str(soup)
#       if not len(soup.split('="review_table'))==1:
        
#         review_soup = soup.split('="review_table')[1].split('row review_table_row')
#         review_soup.pop(0)
#         review_soup = soup.split('="review_table')[1].split('row review_table_row')
#         review_soup.pop(0)
#         for review in review_soup:
                
#                 # extract review
#                 match = re.findall(review_pat, str(review))
#                 if len(match) > 0:
#                     m = match[0]
#                     for iden in ['<div class="the_review"> ','</div>']:
#                         m = m.replace(iden,'')
#                     reviews.append(m.strip('"'))
            

#         # if len(audience_reviews)!=0 :
#         for review in reviews:
#           if review.startswith('<div class="the_review">'):
#             critic_review.append(review.replace('<div class="the_review">','').replace('                                   ','').replace('\n','').replace('                                ','') + '$$')
#             # print(review.replace('<div class="the_review">','').replace('                                   ','').replace('\n',''))
#   return critic_review